In [2]:
import os 
import json
import pprint
%matplotlib inline
from serializer import*
import tweepy
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
from pyvis.network import Network

#Twitter API credentials
api_key = "ENFvcEFBU6prbQWZVMKYHhHCW"
api_secret = "OoLxYiucfTgnheUH1Bp7YJTsyIto9VcxLERBeXCg84NwjjgCAr"
access_token = "2572511777-adhDu1rnH5RYVPdfk51jPzLbOzpppivKmVYTtFH"
access_secret = "Oq1Q8CTYoDKxfyal7aY2x5V88QBqM2SAFac5e8LV6eeWY"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAJkTJQEAAAAAaoRAjF4yb5OBca0iPYQZvs5vH5s%3D5AG5ZaEGM6pFtGL7cQohdtzimqd1aLcqHGbJPOSCXO4Fxnltwz"

In [3]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print ('Authentication completed successfully!')
    
s = Serializer("data")

Authentication completed successfully!


In [ ]:
users = ["mizzaro"]

for user in users:
    print(f"Processing @{user}")

    user_followers = []

    for item in tweepy.Cursor(
            api.friends,
            screen_name=user,
            skip_status=True,
            include_user_entities=False
    ).items():
        user_followers.append(item._json)

    print(f"Found {len(user_followers)} following for @{user}")
    s.serialize_json(f"{user}/{user}_following.json", user_followers)

Processing @mizzaro


Rate limit reached. Sleeping for: 892


In [6]:
import os

directory = "data"
G = nx.DiGraph(membri=["Lorenzo Bellina" "Francesco Bombassei De Bona", "Andrea Cantarutti", "Gabriele Dominici"])
for dirname in os.listdir(directory):
    if not dirname.startswith("."):
        print(f"Analizzo {dirname}")
        s = Serializer(f"data/{dirname}")
        profile = s.read_json(f"{dirname}_profile.json")
        G.add_node(profile["id"], **profile)
        print(f'Aggiungo il nodo {profile["name"]}')

        followers = s.read_json(f"{dirname}_followers.json")
        print(f'Aggiungo i {len(followers)} followers di {profile["name"]}')
        for item in followers:
            G.add_node(item["id"], **item)
            G.add_edge(item["id"], profile["id"])

        following = s.read_json(f"{dirname}_following.json")
        print(f'Aggiungo i {len(following)} following di {profile["name"]}')
        for item in following:
            G.add_node(item["id"], **item)
            G.add_edge(profile["id"], item["id"])

        print(f'Inserisco i random di {profile["name"]}')
        temp_dir = directory+"/"+dirname
        for file in os.listdir(temp_dir):
            id_user = file.split("_")[1]
            if file.startswith("random"):
                if file.endswith("following.json"):
                    following = s.read_json(f"{file}")
                    print(f"Inserisco i {len(following)} following del random di id {id_user}")
                    for item in following:
                        G.add_node(item["id"], **item)
                        G.add_edge(id_user, item["id"])
                elif file.endswith("follower.json"):
                    followers = s.read_json(f"{file}")
                    print(f"Inserisco i {len(followers)} followers del random di id {id_user}")
                    for item in followers:
                        G.add_node(item["id"], **item)
                        G.add_edge(item["id"], id_user)
        node_it = G.in_degree()
        for (node, in_degree) in node_it:
            G.nodes[node]["follower_finded"] = in_degree 
            
nx.write_gpickle(G, "graph/graph_networkx.pkl")

Analizzo damiano10
Data read from path: data/damiano10/damiano10_profile.json
Aggiungo il nodo Damiano Spina
Data read from path: data/damiano10/damiano10_followers.json
Aggiungo i 786 followers di Damiano Spina
Data read from path: data/damiano10/damiano10_following.json
Aggiungo i 836 following di Damiano Spina
Inserisco i random di Damiano Spina
Data read from path: data/damiano10/random_1019221160742014976_follower.json
Inserisco i 10 followers del random di id 1019221160742014976
Data read from path: data/damiano10/random_1037548569467731968_following.json
Inserisco i 10 following del random di id 1037548569467731968
Data read from path: data/damiano10/random_1217315090_following.json
Inserisco i 10 following del random di id 1217315090
Data read from path: data/damiano10/random_135508015_follower.json
Inserisco i 10 followers del random di id 135508015
Data read from path: data/damiano10/random_201194687_follower.json
Inserisco i 10 followers del random di id 201194687
Data read 

In [ ]:
import random
users = ["KevinRoitero", "eglu81"]
for user in users:
    serializer = Serializer(f'data/{user}')
    json = serializer.read_json(f"{user}_followers.json")
    for count in range(0, 5):
        random_follower = random.choice(json)
        random_follower_screenName = random_follower["screen_name"]
        random_follower_id = random_follower["id"]
        random_follower_followers = []
        for item in tweepy.Cursor(
                api.followers,
                screen_name=random_follower_screenName,
                skip_status=True,
                include_user_entities=False
        ).items(10):
            found_follower = item._json
            random_follower_followers.append(found_follower)
        print(f"Found {len(random_follower_followers)} followers for @{random_follower_screenName}")
        serializer.serialize_json(f"random_{random_follower_id}_follower.json", random_follower_followers)

    json = serializer.read_json(f"{user}_following.json")
    for count in range(0, 5):
        random_friend = random.choice(json)
        random_friend_screenName = random_friend["screen_name"]
        random_friend_id = random_friend["id"]
        random_friend_friends = []
        for item in tweepy.Cursor(
                api.friends,
                screen_name=random_friend_screenName,
                skip_status=True,
                include_user_entities=False
        ).items(10):
            found_friend = item._json
            random_friend_friends.append(found_friend)
        print(f"@{random_friend_screenName} follows {len(random_friend_friends)} users")
        serializer.serialize_json(f"random_{random_friend_id}_following.json", random_friend_friends)